In [3]:
n = 1
L = 8*n + 4
t = 1
Ne = Int(L / 2)
function Hamiltonian(L,t)
    H = zeros(2*L, 2*L)
    H[1,L] = -t
    H[L,1] = -t
    H[2L,L+1] = -t
    H[L+1,2L] = -t
    for i=1:2*L
      for j=1:2*L
        if i == j + 1 || i == j - 1
          H[i,j] = -t
        end 
      end
    end
    H[L,L+1]=0
    H[L+1,L]=0
    return H
end

H = Hamiltonian(L,t)

24×24 Array{Float64,2}:
  0.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  -1.0
 

In [4]:
using LinearAlgebra
U = eigvecs(H)  #unitary matrix
Udag = inv(U)   #U^-1
E = eigvals(H)  #eigenvalues of H

24-element Array{Float64,1}:
 -2.0000000000000013
 -1.9999999999999991
 -1.7320508075688772
 -1.732050807568877
 -1.732050807568877
 -1.7320508075688767
 -1.0000000000000002
 -0.9999999999999996
 -0.9999999999999996
 -0.9999999999999996
 -2.2058985591457146e-16
 -3.826984288205141e-18
 -4.253626010865457e-20
  2.2690771583394997e-16
  0.999999999999999
  0.9999999999999991
  0.9999999999999991
  0.9999999999999994
  1.732050807568876
  1.7320508075688767
  1.7320508075688774
  1.7320508075688776
  2.000000000000001
  2.000000000000001

In [5]:
function U1(A, Ne) #take the first Ne columns
    U_1 = A[:,1:Ne]
    return U_1
end

U_1 = U1(U,Ne)

24×6 Array{Float64,2}:
 -0.288675   0.0       -0.204124      0.0           0.353553      0.0
 -0.288675   0.0       -0.353553      0.0           0.204124      0.0
 -0.288675   0.0       -0.408248      0.0           1.84172e-15   0.0
 -0.288675   0.0       -0.353553      0.0          -0.204124      0.0
 -0.288675   0.0       -0.204124      0.0          -0.353553      0.0
 -0.288675   0.0        3.92523e-17   0.0          -0.408248      0.0
 -0.288675   0.0        0.204124      0.0          -0.353553      0.0
 -0.288675   0.0        0.353553      0.0          -0.204124      0.0
 -0.288675   0.0        0.408248      0.0           1.92023e-15   0.0
 -0.288675   0.0        0.353553      0.0           0.204124      0.0
 -0.288675   0.0        0.204124      0.0           0.353553      0.0
 -0.288675   0.0        0.0           0.0           0.408248      0.0
  0.0       -0.288675   0.0          -0.204124      0.0           0.353553
  0.0       -0.288675   0.0          -0.353553      0.0       

In [6]:
function occupied_states(v)
    occ_states = []
    for i = 1 : 2*L
        if v[i] == 1
            append!(occ_states, i )
        end
    end
    return occ_states
end
    

function U_tilde(A,v)                  #takes just the rows related to occupied states, Ne*Ne matrix
    U_tild = []
    U_tild = A[occupied_states(v),:]
    return U_tild
end


U_tilde (generic function with 1 method)

In [2]:
state = [1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1]      #initial state

U_2 = U_tilde(U_1,state)                       #initial U tilde

state_1 = [1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1]    #|x'>

U_3 = U_tilde(U_1,state_1)                     #new U tilde

ratio = det(U_3) / det(U_2)                    #ratio of determinants of U tilde matrices

LoadError: [91mUndefVarError: U_1 not defined[39m

In [140]:
#Jastrow factor
Jastrow_ratio = 1


1

In [48]:
function n_i(vi)
    
    n_up = 0
    n_down = 0
    
        if vi == 1
            n_up += 1
            
        elseif vi == -1
            n_down += 1
                
        elseif vi == 2
            n_down += 1
            n_up += 1
                
        end

return n_up, n_down

end


n_i (generic function with 1 method)

In [25]:
v=2

 n_i(v)

(1, 1)

In [20]:
function Jastrow(state,g,v)
            
    D=length(state)        
    
    n1 = 1
    n2 = 0
    
    Spins=0
    Occupancies=0
            
    n1 = n_i(state[D])[1] + n_i(state[D])[2]  
    
    for i = 1 : D
                
        n_up= n_i(state[i])[1]
                
        n_down = n_i(state[i])[2]
                
             
    Spins = Spins + n_up*n_down 
                
        
    n2= n_up + n_down
    n1n2 = n1*n2   
    n1 = n2
                
    Occupancies = Occupancies + n1n2
            
        end
    
    arg=  -g*Spins -v*Occupancies
            
            return exp(arg)
    
    end


Jastrow (generic function with 1 method)

In [22]:
state = [1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1] 

Jastrow(state,1,1)

0.1353352832366127

In [ ]:
WaveFunction=  U_tilde(U_1,state) 

In [7]:
function MC(q0=0,delta=0.1,nsteps=1000,seed=None):
    if seed is not None:
        np.random.seed(seed)
    q=q0
    traj=[]
    for istep in range(nsteps):
        qtry=q+(2*np.random.rand()-1)*delta
        energy_try=0.5*kappa*qtry**2     #1/2 k q^2
        deltae=energy_try-energy
        if deltae<=0.0:          #take acceptance 1 if the energy decrease
            acceptance=1.0
        else:
            acceptance =  ratio**2 * Jastrow_ratio**2   #acceptance 
        if acceptance >=1.0 or acceptance>np.random.rand(): #metropolis rule
            q=qtry
            energy=energy_try
        traj.append(q)
    return np.array(traj)

LoadError: [91msyntax: newline not allowed after ":" used for quoting[39m

In [8]:
state = [1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1]      #initial state
function move(v1)                              #generate a random move from a random occupied state
    v = copy(v1)
    len_v = length(v)
    x = occupied_states(v)
    n = length(x)
    i = rand(1:n)
    v[x[i]] = 0
    
    if 1 < x[i] < length(v)
        v[x[i]+2*rand(0:1)-1] += 1             #random +-1 move if not at boundaries
    elseif x[i]==1                             #if in position 1 move either to 2 or to n
        if 2*rand(0:1)-1 < 0
            v[len_v] += 1
        else
            v[2] += 1
        end
    else 
        if 2*rand(0:1)-1 < 0                    #if in position n move either to n-1 or to 1
            v[len_v-1] += 1
        else
            v[1] += 1
        end 
    end
    return v
end

move (generic function with 1 method)

In [105]:
function MC
    state_new = move(state)   
    acceptance =  ratio**2 * Jastrow_ratio**2       #acceptance 
    if acceptance >=1.0 or acceptance>rand()        #metropolis rule
        state = copy(state_new)
        #update T jastrow
    end
    
        

LoadError: syntax: expected "end" in definition of function "MC"

In [15]:
#generation of random state
#This is not taking into account double occupancies
using StatsBase
function randomstate(L)
    state = sample(vcat([1 for j=1:L/4],[-1 for j=1:L/4],[0 for j=1:L/2]), L, replace=false) #sample randomly assigns values of the vector concatenation vcat to a vector of length L without replacement
    return state
end

randomstate (generic function with 1 method)

In [35]:
#Average electron density
function averagedensity(L)
nprom = 2/L sum()

[-1, 1, 0, 1, -1, 0, 0, 0, 1, 0, -1, 0][-1, 1, 1, 1, -1, -1, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, -1, 0][-1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, -1, 1, 1, 0, 0, 0, 0, 1, 0, -1, -1, 0, -1, 1, -1, 1][1, 0, 0, -1, -1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, -1, 1, -1, 1, 0, 1, -1, 0, -1, 1, 0, 1, 1][0, 0, 0, -1, 0, -1, 0, -1, -1, 0, 0, 1, 1, 0, 0, 1, -1, 0, 0, -1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, 0, 0, 1, 1, -1, 0, 1, 0, 0, 0, 1, 0, 1, 1][0, 1, -1, -1, 0, 0, 0, -1, 0, 0, 1, 1, 0, -1, -1, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, 1, 0, 1, -1, -1, 0, 1, 0, -1, -1, 1, 1, 1, 1, -1, 1, 0, 0, 0][0, 0, 1, 0, 1, -1, 1, 0, 1, 1, 1, -1, 0, 0, -1, 0, -1, -1, 1, 0, -1, 0, 0, 1, -1, -1, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, -1, 0, 1, 1, 0, 0, -1, -1, 1][1, -1, 1, 1, -1, 0, 0, 1, -1, 1, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, -1, -1, 0, 0, -1, 1, 1, 0, -1, 1, 0, 0, 1, 0, -1, 0, 0, 0, -1, -1, -1, 0, -1

In [41]:
v = [1,2,3,4,5]
j = 1
if j%2 != 0
    sum(v[j] for j=1:length(v))
end

15

In [65]:
#This function calculates the density observable in the odd sites A
function nA(v)
    nA = 0
    for j=1:length(v)
        if j%2 != 0
            nA += sum(n_i(v[j])) 
        end
    end
    return nA*2/length(v)
end

#This function calculates the density observable in the even sites B
function nB(v)
    nB = 0
    for j=1:length(v)
        if j%2 == 0
            nB += sum(n_i(v[j])) 
        end
    end
    return nB*2/length(v)
end

nB (generic function with 1 method)

In [66]:
nA(randomstate(12))
nB(randomstate(12))

0.3333333333333333

In [ ]:
function averagedensity(state)
    nprom = oddsum()